### Transit Scores from Census Block Groups
This notebook contains code to calculate transit and walk scores for each Census Tract in San Diego County. The general approach is as follows:
* get centroid latitude/longitude points for each Census Block Group
* get population count for each Census Block Group
* get walkscore and transit score for each centroid location using the [WalkScore.com](https://www.walkscore.com/) API
* create a walkscore and transit score for each Census Tract by weighting the walkscore/transit scores of all Block Groups by Block Group population and then averaging
* save the weighted scores per Census Tract for use in the county map

Population counts per Census Block Group are estimates from the 2017 American Community Survey.

In [1]:
import shapefile
import json
import pandas as pd
import numpy as np
import requests

#### Create dataframe with Census Block Group centroid information
The shapefile can be found at https://www.census.gov/cgi-bin/geo/shapefiles/index.php
* Select year 2018
* Select Block Groups
* Select State (California)

Centroid latitude and longitude are stored as 'INTPTLAT' and 'INTPTLON'

In [2]:
SANDIEGO_FIPS = "073"  # Code for San Diego County
shp_path = "data_blockgroups/tl_2018_06_bg.shp"
sf = shapefile.Reader(shp_path)

fields = [x[0] for x in sf.fields][1:]

records = sf.records()
num_records = len(records)

tmp_list = []
for i in range(num_records):
    record = sf.record(i)
    tmp_list.append(record[0:15])

df_centroids = pd.DataFrame(tmp_list, columns=fields)
df_centroids = df_centroids.drop(['MTFCC','ALAND','AWATER','FUNCSTAT'],axis=1)
df_centroids = df_centroids[df_centroids['COUNTYFP']==SANDIEGO_FIPS]
df_centroids.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,INTPTLAT,INTPTLON
194,06,073,003603,1,060730036031,Block Group 1,+32.6871318,-117.1103976
346,06,073,017106,3,060730171063,Block Group 3,+33.0343151,-117.2103363
347,06,073,013908,2,060730139082,Block Group 2,+32.7111349,-117.0071494
792,06,073,020903,1,060730209031,Block Group 1,+33.3304770,-116.7941963
793,06,073,021302,1,060730213021,Block Group 1,+32.6209574,-116.8074319


#### Create dataframe with block population information
This data can be found at https://factfinder2.census.gov/
* Select Advanced Search (show me all)
* Select Topics -> People -> Population Total
* Select Geographies -> State/County/Block Group
    * Select State -> California
    * Select County -> San Diego
* Choose the TOTAL POPULATION (2017 ACS 5-year estimates) table to download

I edited the downloaded csv file to remove the extra header, and resaved it as 'pop_estimates_2017_bg.csv'

In [3]:
df_pop = pd.read_csv("data_blockgroups/pop_estimates_2017_bg.csv", index_col=0)
df_pop = df_pop.reset_index()
df_pop['Id2'] = df_pop['Id2'].apply(str)
df_pop['Id2'] = '0' + df_pop['Id2']
df_pop.head()

,Id,Id2,Geography,EstimateTot,Margin of Error
0,1500000US060730001001,060730001001,"Block Group 1, Census Tract 1, San Diego Count...",1276,157
1,1500000US060730001002,060730001002,"Block Group 2, Census Tract 1, San Diego Count...",1587,185
2,1500000US060730002011,060730002011,"Block Group 1, Census Tract 2.01, San Diego Co...",2056,175
3,1500000US060730002021,060730002021,"Block Group 1, Census Tract 2.02, San Diego Co...",1521,453
4,1500000US060730002022,060730002022,"Block Group 2, Census Tract 2.02, San Diego Co...",713,373


#### Join dataframes and save intermediate file to CSV

In [4]:
df_bg = pd.merge(df_centroids, df_pop, left_on='GEOID', right_on='Id2')
df_bg = df_bg.drop(['NAMELSAD','Id','Id2','Geography','Margin of Error'], axis=1)
df_bg.to_csv('data/BlockGroupsWithCentroidsPop.csv', header=True)
df_bg.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,INTPTLAT,INTPTLON,EstimateTot
0,06,073,003603,1,060730036031,+32.6871318,-117.1103976,2348
1,06,073,017106,3,060730171063,+33.0343151,-117.2103363,1149
2,06,073,013908,2,060730139082,+32.7111349,-117.0071494,3629
3,06,073,020903,1,060730209031,+33.3304770,-116.7941963,334
4,06,073,021302,1,060730213021,+32.6209574,-116.8074319,5239


#### Get walkscore and transit score for each Census block
This section requires the [WalkScore.com API](https://www.walkscore.com/professional/api.php). The free API is limited to 5,000 calls per day, and there are 1,795 block groups in San Diego County. This code block took about 45 min to run. A csv file with the walk/transit scores ('BlockGroupWithTransit.csv') is provided on this repository, and is preloaded here.

Calls to the Walkscore.com API need to be formatted as follows (all one line):

`url = 'http://api.walkscore.com/score?format=json&lat=47.6085&lon=-122.3295&transit=1&wsapikey=YOUR_API_KEY'`

where you replace the example lat and long values with your own.

In [5]:
#begin_str = 'http://api.walkscore.com/score?format=json'
#end_str = '&transit=1&wsapikey=YOUR_API_KEY'
#
#blocks_walk = []
#blocks_transit = []
#
#for index, row in df_bg.iterrows():
#    mid_str = '&lat=' + str(row.INTPTLAT) + '&lon=' + str(row.INTPTLON)
#    url = begin_str + mid_str + end_str
#    r = requests.get(url)
#    json_data = r.json()
#    if ('walkscore' in json_data) and ('transit' in json_data):
#        blocks_walk.extend([json_data['walkscore']])
#        blocks_transit.extend([json_data['transit']['score']])
#    elif ('walkscore' in json_data) and ('transit' not in json_data):
#        if (json_data['walkscore']==0):
#            blocks_walk.extend([json_data['walkscore']])
#            blocks_transit.extend([0])
#        else:
#            blocks_walk.extend([json_data['walkscore']])
#            blocks_transit.extend([np.nan])            
#    elif ('walkscore' not in json_data) and ('transit' in json_data):
#        blocks_walk.extend([np.nan])
#        blocks_transit.extend([json_data['transit']['score']])
#    else:
#        blocks_walk.extend([np.nan])
#        blocks_transit.extend([np.nan])
#
#df_bg['BLOCK_WALK'] = pd.Series(blocks_walk)
#df_bg['BLOCK_TRANSIT'] = pd.Series(blocks_transit)
#df_bg.to_csv('data/BlockGroupWithTransit.csv', header=True)

In [6]:
df_bg = pd.read_csv("data/BlockGroupWithTransit.csv", index_col=0)
df_bg.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,INTPTLAT,INTPTLON,EstimateTot,BLOCK_WALK,BLOCK_TRANSIT
0,6,73,3603,1,60730036031,32.687132,-117.110398,2348,56.0,47.0
1,6,73,17106,3,60730171063,33.034315,-117.210336,1149,0.0,0.0
2,6,73,13908,2,60730139082,32.711135,-117.007149,3629,74.0,NaN
3,6,73,20903,1,60730209031,33.330477,-116.794196,334,NaN,NaN
4,6,73,21302,1,60730213021,32.620957,-116.807432,5239,0.0,0.0


#### Calculate weighted walk/transit scores across each Census Tract
Only block groups with a walkscore (or transit score) are included in the total population count for that tract. This means that block groups with no walkscore (or transit score) have no impact on the overall tract score, but that also may skew the weighting among the remaining block groups.

The file created here ('CensusTract_Transit_BlockGroups.csv') is used in the RASP_Tract notebook.

In [8]:
# Convert Tract code to strings with uniform length
df_bg['TRACTCE'] = df_bg['TRACTCE'].apply(str)

mask = (df_bg['TRACTCE'].str.len() == 4)
df_bg.loc[mask, 'TRACTCE'] = '00' + df_bg.loc[mask, 'TRACTCE']

mask = (df_bg['TRACTCE'].str.len() == 5)
df_bg.loc[mask, 'TRACTCE'] = '0' + df_bg.loc[mask, 'TRACTCE']

tracts = df_bg.TRACTCE.unique()
prefix = '06073'

# Calculate weighted scores across each Census tract
tract_list = []
for tract in tracts:
    df_bg.loc[(df_bg['TRACTCE']==tract)&(~df_bg['BLOCK_WALK'].isna()) , 'EstimateTot']
    tot_pop = df_bg.loc[(df_bg['TRACTCE']==tract)&(~df_bg['BLOCK_WALK'].isna())].EstimateTot.sum()
    tot_walk = (df_bg.loc[(df_bg['TRACTCE']==tract)&(~df_bg['BLOCK_WALK'].isna())].EstimateTot
                * df_bg.loc[(df_bg['TRACTCE']==tract)&(~df_bg['BLOCK_WALK'].isna())].BLOCK_WALK).sum()
    weight_walk = tot_walk/tot_pop

    df_bg.loc[(df_bg['TRACTCE']==tract)&(~df_bg['BLOCK_TRANSIT'].isna()) , 'EstimateTot']
    tot_pop = df_bg.loc[(df_bg['TRACTCE']==tract)&(~df_bg['BLOCK_TRANSIT'].isna())].EstimateTot.sum()
    tot_transit = (df_bg.loc[(df_bg['TRACTCE']==tract)&(~df_bg['BLOCK_TRANSIT'].isna())].EstimateTot
                * df_bg.loc[(df_bg['TRACTCE']==tract)&(~df_bg['BLOCK_TRANSIT'].isna())].BLOCK_TRANSIT).sum()
    weight_transit = tot_transit/tot_pop

    geoid2 = prefix + tract
    tract_list.append([geoid2, weight_walk, weight_transit])
    
# Create new dataframe of Tract values
blocks_transit = pd.DataFrame(tract_list, columns=['GEOID','TRACT_WALK','TRACT_TRANSIT'])
blocks_transit.to_csv('data/CensusTract_Transit_BlockGroup.csv', header=True)
blocks_transit.head()

/Users/marissawesterfield/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars
/Users/marissawesterfield/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars


,GEOID,TRACT_WALK,TRACT_TRANSIT
0,06073003603,58.192326,46.766391
1,06073017106,0.665722,0.000000
2,06073013908,70.629787,NaN
3,06073020903,0.000000,0.000000
4,06073021302,0.000000,0.000000
